*Автор: Татьяна Рогович*

# Анализ данных в Python

## Реализация класса kNN

*Автор: Татьяна Рогович, НИУ ВШЭ*

In [39]:
import pandas as pd

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"

# Назовем колонки датасета
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'Class']

# создадим датасет
dataset = pd.read_csv(url, names=names)
dataset

,sepal-length,sepal-width,petal-length,petal-width,Class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [40]:
X = dataset[['sepal-length', 'sepal-width']].values
y = dataset['Class'].values

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Реализация kNN в виде класса

У нас уже есть все нужные нам кирпичики, поэтому реализуем kNN в виде класса. Он у нас будет состоять из конструктора, где мы инициализруем переменные, метода `dist`, который возвращает евклидово расстояние, метода `fit`, который будет задавать переменные обучающей выборки, и метода `predict`, который будет считать дистанции возвращать нам предсказанные значения. Также создадим метод `score`, считающуй accuracy,  и метод `plot`, который будет рисовать графики.

In [41]:
class KNN:
    """
    k-NN классификатор

    Возвращает: предсказания k-NN
    """

    def __init__(self, k):
        self.X_train = None
        self.y_train = None
        self.k = k
        self.predictions = []

Конструктор у нас будет просто инициализировать переменные. Как правило в ML мы создаем экземпляр класса не на данных, а с определенными параметрами (например, задаем количество соседей). А данные будем уже передавать методу .fit() (обучение). Чтобы обучать нам сначала нужно определить метод, считающий расстояние. Т.к. этот метод не принимает данные напрямую, а будет только вызываться в predict(), то объявим его статическим. Также "скроем" его нижним подчеркиванием (намекнем, что мы не предполагаем, что этот метод должен вызываться от объекта класса).

In [42]:
class KNN:
    """
    k-NN классификатор

    Возвращает: предсказания k-NN
    """

    def __init__(self, k):
        self.X_train = None
        self.y_train = None
        self.k = k
        self.predictions = []
    
    @staticmethod
    def _dist(a, b):
        """
        Расстояние евклида
        Принимает на вход два вектора

        Возвращает: число с плавающей точкой
        """
        return ((a[0] - b[0])**2 + (a[1] - b[1])**2)**0.5

In [43]:
test = KNN(2)
test._dist([2,4],[4,2]) # но проверить, что работает можем!

2.8284271247461903

Теперь давайте определим метод .fit(). Это уже будет динамический метод, который вызывается от экземпляра класса. fit() должен сохранять тренировочные выборки в нужные атрибуты класса.

In [44]:
class KNN:
    """
    k-NN классификатор

    Возвращает: предсказания k-NN
    """

    def __init__(self, k):
        self.X_train = None
        self.y_train = None
        self.k = k
        self.predictions = []
    
    @staticmethod
    def _dist(a, b):
        """
        Расстояние евклида
        Принимает на вход два вектора

        Возвращает: число с плавающей точкой
        """
        return ((a[0] - b[0])**2 + (a[1] - b[1])**2)**0.5
    
    def fit(self, X_train, y_train, k):
        """
        Принимает на вход два массива с данными: тренировочный Х и тренировочные лейблы
        """
        self.X_train = X_train
        self.y_train = y_train

Теперь реализуем predict() - здесь уже будет происходить много всего. Метод будет принимать тестовые точки и находить расстояние от них всех до тренировочных точек. Потом будет брать k соседей и предсказывать по ним класс объекта.

In [45]:
class KNN:
    """
    k-NN классификатор

    Возвращает: предсказания k-NN
    """

    def __init__(self, k):
        self.X_train = None
        self.y_train = None
        self.k = k
        self.predictions = []
    
    @staticmethod
    def _dist(a, b):
        """
        Расстояние евклида
        Принимает на вход два вектора

        Возвращает: число с плавающей точкой
        """
        return ((a[0] - b[0])**2 + (a[1] - b[1])**2)**0.5
    
    def fit(self, X_train, y_train):
        """
        Принимает на вход два массива с данными: тренировочный Х и тренировочные лейблы
        """
        self.X_train = X_train
        self.y_train = y_train
        
    def predict(self, X_test):
        """
        Принимает на вход двумерный массив искомых точек

        Возвращает: список предсказаний
        """
        
        for i in range(len(X_test)):
            distances = []
            targets = {}

            for j in range(len(X_train)):
                # пройдем по всем точкам и посчитаем расстояние до них от тестовой точки
                distances.append([self._dist(X_test[i], X_train[j]), j])

            # отсортируем расстояния
            distances = sorted(distances)

            # создадим словарь с k ближайщими значениями
            for j in range(self.k):
                index = distances[j][1]
                if targets.get(y_train[index]) != None:
                    targets[y_train[index]] += 1
                else:
                    targets[y_train[index]] = 1

            # вернем самую часто встречающаюся метку
            self.predictions.append(max(targets,key=targets.get))

        return self.predictions

Давайте попробуем эти методы в деле.

In [46]:
knn = KNN(7)
knn.fit(X_train,y_train)
pred = knn.predict(X_test)
print(pred)

['Iris-versicolor', 'Iris-setosa', 'Iris-virginica', 'Iris-versicolor', 'Iris-versicolor', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-versicolor', 'Iris-virginica', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-virginica', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-virginica', 'Iris-setosa', 'Iris-versicolor', 'Iris-setosa', 'Iris-virginica', 'Iris-virginica', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-setosa', 'Iris-setosa']


In [47]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn_sklearn = KNeighborsClassifier(7, p=2)
knn_sklearn.fit(X_train,y_train)
pred_sklearn = knn_sklearn.predict(X_test)
print(accuracy_score(pred, y_test))
print(accuracy_score(pred_sklearn, y_test))

0.7333333333333333
0.7666666666666667


## Класс kNN с другой функций расстояния

Давайт создадим второй класс kNN, который теперь использует другую функцию расстояния. Создавать второй класс с нуля не будем, а воспользуемся свойством классов - наследование. В новой классе поменяем только функцию расстояния, а все остальное оставим таким же. Для нахождения косинусной меры, используем функцию `cosine` из библиотеки `scipy`.

![](https://datascience-enthusiast.com/figures/cosine_sim.png)

In [48]:
from scipy.spatial import distance

class cosKNN(KNN):
    
    @staticmethod
    def _dist(a,b):
        """
        Через косинусную меру

        возвращаетs: число с плавающей точкой
        """
        return distance.cosine(a,b)

In [49]:
cos_kNN = cosKNN(7)
cos_kNN.fit(X_train,y_train)
pred = cos_kNN.predict(X_test)
print(pred)
print(accuracy_score(pred, y_test))

['Iris-versicolor', 'Iris-setosa', 'Iris-virginica', 'Iris-versicolor', 'Iris-virginica', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-versicolor', 'Iris-versicolor', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-virginica', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-setosa', 'Iris-versicolor', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-versicolor', 'Iris-setosa', 'Iris-setosa']
0.6666666666666666


## Класс kNN с поддержкой нескольких функций расстояния

Выше мы создали два класса, один из которых наследовался с новой функцией расстояния (метрикой). Но это все можно было реализовать в рамках нашего одного базового класса. Давайте применим все наши знания о классах и создадим его,также модицифируем метод `dist` (добавим функционал выбора метрики, которую будем передавать в конструкторе).

In [50]:
class ultimateKNN:
    """
    k-NN классификатор

    Возвращает: предсказания k-NN
    """

    def __init__(self, k=3, metric='eucl'): # зададим соседей и метрики по умолчанию
        self.X_train = None
        self.y_train = None
        self.k = k
        self.metric = metric
        self.predictions = []
    
    def _dist(self, a, b):
        """
        Расстояние Евклида или косинусная мера
        Принимает на вход два вектора

        Возвращает: число с плавающей точкой
        """
        if self.metric == "eucl":
            return ((a[0] - b[0])**2 + (a[1] - b[1])**2)**0.5
        elif self.metric == "cos":
            return distance.cosine(a,b)
    
    
    def fit(self, X_train, y_train):
        """
        Принимает на вход два массива с данными: тренировочный Х и тренировочные лейблы
        """
        self.X_train = X_train
        self.y_train = y_train
        
    def predict(self, X_test):
        """
        Принимает на вход двумерный массив искомых точек

        Возвращает: список предсказаний
        """
        self.X_test = X_test
        
        for i in range(len(self.X_test)):
            distances = []
            targets = {}

            for j in range(len(self.X_train)):
                # пройдем по всем точкам и посчитаем расстояние до них от тестовой точки
                distances.append([self._dist(self.X_test[i], self.X_train[j]), j])

            # отсортируем расстояния
            distances = sorted(distances)

            # создадим словарь с k ближайщими значениями
            for j in range(self.k):
                index = distances[j][1]
                if targets.get(self.y_train[index]) != None:
                    targets[self.y_train[index]] += 1
                else:
                    targets[self.y_train[index]] = 1

            # вернем самую часто встречающаюся метку
            self.predictions.append(max(targets,key=targets.get))

        return self.predictions

In [51]:
ult_kNN = ultimateKNN(7,metric='cos')
ult_kNN.fit(X_train,y_train)
print(ult_kNN.predict(X_test))

['Iris-versicolor', 'Iris-setosa', 'Iris-virginica', 'Iris-versicolor', 'Iris-virginica', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-versicolor', 'Iris-versicolor', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-versicolor', 'Iris-virginica', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-setosa', 'Iris-versicolor', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-versicolor', 'Iris-setosa', 'Iris-setosa']


In [52]:
ult_kNN = ultimateKNN(7)
ult_kNN.fit(X_train,y_train)
print(ult_kNN.predict(X_test))

['Iris-versicolor', 'Iris-setosa', 'Iris-virginica', 'Iris-versicolor', 'Iris-versicolor', 'Iris-setosa', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-versicolor', 'Iris-virginica', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-setosa', 'Iris-virginica', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-virginica', 'Iris-setosa', 'Iris-versicolor', 'Iris-setosa', 'Iris-virginica', 'Iris-virginica', 'Iris-versicolor', 'Iris-versicolor', 'Iris-virginica', 'Iris-setosa', 'Iris-setosa']


Давайте теперь сделаем сетку - построим несколько кнн классификаторов с разными `k` и посмотрим какой из них круче.

In [53]:
for k in range(1,10):
    knn = ultimateKNN(k, metric='cos')
    knn.fit(X_train,y_train)
    pred = knn.predict(X_test)
    print("k = " + str(k), ", Score: " + str(accuracy_score(pred, y_test)))

k = 1 , Score: 0.7
k = 2 , Score: 0.7
k = 3 , Score: 0.6666666666666666
k = 4 , Score: 0.6
k = 5 , Score: 0.6333333333333333
k = 6 , Score: 0.6666666666666666
k = 7 , Score: 0.6666666666666666
k = 8 , Score: 0.7333333333333333
k = 9 , Score: 0.6666666666666666


In [54]:
for k in range(1,10):
    knn = ultimateKNN(k)
    knn.fit(X_train,y_train)
    pred = knn.predict(X_test)
    print("k = " + str(k), ", Score: " + str(accuracy_score(pred, y_test)))

k = 1 , Score: 0.8
k = 2 , Score: 0.8
k = 3 , Score: 0.8
k = 4 , Score: 0.8666666666666667
k = 5 , Score: 0.8
k = 6 , Score: 0.8666666666666667
k = 7 , Score: 0.7333333333333333
k = 8 , Score: 0.8
k = 9 , Score: 0.8
